In [118]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as datetime

from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier, StackingClassifier, RandomForestClassifier, BaggingClassifier

from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support, balanced_accuracy_score, plot_precision_recall_curve, precision_recall_curve, precision_score, recall_score

from sklearn.model_selection import RandomizedSearchCV
import shap

In [70]:
df = pd.read_csv("Pipes_Break20.csv")

print(df.columns)
# # select non join columns and ML needed columns
columns = ['OBJECTID', 'DWW_Mainlines__Permitted_Use__MNL_FEAT_1', 'DWW_Mainlines__Permitted_Use__MNL_MATE_1', 
       'DWW_Mainlines__Permitted_Use__MNL_LENGTH', 'DWW_Mainlines__Permitted_Use__MNL_INSTAL',
       'Pipe_widths_Width', 'NEAR_DIST', 'MUSYM', 'ARTCLASS', 'BLOCKNBR',
       'SPEEDLIMIT', 'SURFACEWID', 'SURFACETYP', 'SLOPE_PCT', 'Size', 'DATE']
non_joins = df[columns]
non_joins.head(10)

Index(['OBJECTID', 'Join_Count', 'Join_Count_1',
       'FID_DWW_Mainlines__Permitted_Use__Pipe_widths',
       'DWW_Mainlines__Permitted_Use__MNL_FEA_KE',
       'DWW_Mainlines__Permitted_Use__MNL_FEAT_1',
       'DWW_Mainlines__Permitted_Use__MNL_OWNE_1',
       'DWW_Mainlines__Permitted_Use__MNL_MATE_1',
       'DWW_Mainlines__Permitted_Use__MNL_LENGTH',
       'DWW_Mainlines__Permitted_Use__MNL_INSTAL', 'Pipe_widths_MNL_FEA_KEY',
       'Pipe_widths_Width', 'FID_SeaSoilClip', 'MUSYM', 'MUKEY', 'muname',
       'ARTCLASS', 'UNITDESC', 'STNAME_ORD', 'BLOCKNBR', 'SPEEDLIMIT',
       'SURFACEWID', 'SURFACETYP', 'SLOPE_PCT', 'NEAR_DIST', 'NEAR_X',
       'NEAR_Y', 'WONUM', 'ASSETNUM', 'Size', 'Long', 'Lat', 'DATE'],
      dtype='object')


,OBJECTID,DWW_Mainlines__Permitted_Use__MNL_FEAT_1,DWW_Mainlines__Permitted_Use__MNL_MATE_1,DWW_Mainlines__Permitted_Use__MNL_LENGTH,DWW_Mainlines__Permitted_Use__MNL_INSTAL,Pipe_widths_Width,NEAR_DIST,MUSYM,ARTCLASS,BLOCKNBR,SPEEDLIMIT,SURFACEWID,SURFACETYP,SLOPE_PCT,Size,DATE
0,1,Mainline,Concrete,314.79,1/1/1972 0:00:00,8.0,3964.600720,3055,2.0,9400.0,25.0,40.0,PCC,0.0,NaN,NaN
1,2,Mainline,Concrete,363.39,1/1/1972 0:00:00,8.0,3609.210948,3056,0.0,0.0,20.0,46.0,AC,4.0,NaN,NaN
2,3,Mainline,Concrete,323.51,1/1/1972 0:00:00,8.0,3451.254435,3056,0.0,0.0,20.0,46.0,AC,4.0,NaN,NaN
3,4,Mainline,Vitrified Clay,329.13,1/1/1928 0:00:00,12.0,833.915482,3056,0.0,5100.0,20.0,0.0,ST,6.0,NaN,NaN
4,5,Mainline,Reinforced Concrete Pipe,273.64,1/1/1928 0:00:00,18.0,852.426502,3056,2.0,10300.0,25.0,42.0,AC/PCC,4.0,NaN,NaN
5,6,Mainline,Concrete,30.87,1/1/1963 0:00:00,8.0,1404.924372,3056,0.0,400.0,20.0,23.0,AC,3.0,NaN,NaN
6,7,Mainline,Reinforced Concrete Pipe,112.29,1/1/1990 0:00:00,18.0,2212.976394,988,0.0,1200.0,20.0,22.0,ST,2.0,NaN,NaN
7,8,Stub,Concrete,29.89,1/1/1958 0:00:00,8.0,3065.662515,3055,0.0,2200.0,20.0,21.0,ST,1.0,NaN,NaN
8,9,Stub,Concrete,7.49,1/1/1958 0:00:00,8.0,3148.924667,3055,0.0,2300.0,20.0,21.0,ST,2.0,NaN,NaN
9,10,Mainline,Concrete,42.59,1/1/1958 0:00:00,8.0,3196.074038,3059,0.0,200.0,20.0,18.0,ST,11.0,NaN,NaN


In [71]:
ddff = non_joins[non_joins['DATE'].notna()]
print(len(ddff['DATE']))
print(len(ddff['DATE'].unique()))
print('duplicate breaks: ', len(ddff['DATE']) - len(ddff['DATE'].unique()))

2800
1951
duplicate breaks:  849


In [72]:

# This line removes duplicate breaks, based on the date column. 
non_joins = non_joins[(~non_joins['DATE'].duplicated() | df['DATE'].isnull())]



In [73]:
# select only years
non_joins['DWW_Mainlines__Permitted_Use__MNL_INSTAL'] = pd.to_datetime(non_joins['DWW_Mainlines__Permitted_Use__MNL_INSTAL'], format='%m/%d/%Y %H:%M:%S')
non_joins['DWW_Mainlines__Permitted_Use__MNL_INSTAL'] = non_joins['DWW_Mainlines__Permitted_Use__MNL_INSTAL'].map(lambda x: x.year)

non_joins['DATE'] = pd.to_datetime(non_joins['DATE'], format='%m/%d/%Y %H:%M:%S')
non_joins['DATE'] = non_joins['DATE'].map(lambda x: x.year)

non_joins.head()

,OBJECTID,DWW_Mainlines__Permitted_Use__MNL_FEAT_1,DWW_Mainlines__Permitted_Use__MNL_MATE_1,DWW_Mainlines__Permitted_Use__MNL_LENGTH,DWW_Mainlines__Permitted_Use__MNL_INSTAL,Pipe_widths_Width,NEAR_DIST,MUSYM,ARTCLASS,BLOCKNBR,SPEEDLIMIT,SURFACEWID,SURFACETYP,SLOPE_PCT,Size,DATE
0,1,Mainline,Concrete,314.79,1972,8.0,3964.600720,3055,2.0,9400.0,25.0,40.0,PCC,0.0,NaN,NaN
1,2,Mainline,Concrete,363.39,1972,8.0,3609.210948,3056,0.0,0.0,20.0,46.0,AC,4.0,NaN,NaN
2,3,Mainline,Concrete,323.51,1972,8.0,3451.254435,3056,0.0,0.0,20.0,46.0,AC,4.0,NaN,NaN
3,4,Mainline,Vitrified Clay,329.13,1928,12.0,833.915482,3056,0.0,5100.0,20.0,0.0,ST,6.0,NaN,NaN
4,5,Mainline,Reinforced Concrete Pipe,273.64,1928,18.0,852.426502,3056,2.0,10300.0,25.0,42.0,AC/PCC,4.0,NaN,NaN


In [74]:
# filter out NaN from DATE col
breaks_df = non_joins[non_joins['DATE'].notna()]
# set size column to width column and drop size 
breaks_df['Pipe_widths_Width'] = breaks_df['Size']
breaks_df = breaks_df.drop('Size', axis=1)
# breaks_df.head()

## ML Pseudo Steps
#### 1) assign binary variable --> can be raw data set
#### 2) create dummy variables for categorical columns
* make sure soil column ['MUSYM'] is categorical
#### 3) list of years (and cycle through those)
#### 4) will have six year groups :
* train on [2009, 2010, 2011], test on [2012, 2013, 2014]
* move onto [2010, 2011, 2012], test on [2013, 2014, 2015], etc ...
#### 5) create function to create subset of data
* want to include where there is NOT a break year (those will be our non-broken positive examples)
* want to include where break year is in time frame of what we want
* exclude installs AFTER time frame window
* based on time window, calculate appropriate age of pipes 
    * (select beginning year of time frame --> ex: [2009, 2010, 2011] subtract install year from 2009)


In [75]:
pseudo_df = non_joins
# pseudo_df.head()

# fill Nan with 0 (idk why, but it just made it work FOR NOW)
# if date is not 0 (NaN), make width = size 
# then for all df, drop size column

pseudo_df['DATE'] = pseudo_df['DATE'].fillna(0)
pseudo_df.loc[pseudo_df['DATE'] != 0, ['Pipe_widths_Width']] = pseudo_df['Size']
pseudo_df = pseudo_df.drop('Size', axis=1)
# pseudo_df.head()

# change 'Width' values to numbers instead of strings (for dummy prep)
pseudo_df['Width'] = pd.to_numeric(pseudo_df['Pipe_widths_Width'], errors='coerce')
# pseudo_df['Width'].unique()

# Assign binary variables:
# [Create new column] If pipe has a broken date --> broken pipes = 0, non-broken pipes = 1
pseudo_df['TARGET'] = pseudo_df['DATE'].apply(lambda x: 0 if x == 0 else 1)

In [76]:
# Create dummy variables
dummy_df = pd.get_dummies(pseudo_df)
# test_df.columns

In [77]:
def get_data(df, start, end):
    """
    Takes in df and filters depending on timeframe (start and end years).
    Returns subset of data for training in timeframe.
    """
    # want to include where there is NOT a break year (those will be our non-broken positive examples --> 'DATE' == 0) - DATE col
    # want to include where break year is in time frame of what we want - DATE col
    train_df = df[(df['DATE'] == 0 )| ((df['DATE'] >= start) & (df['DATE'] <= end))]

    # exclude installs AFTER time frame window - MNL_INSTAL col
    train_df = train_df[(train_df['DWW_Mainlines__Permitted_Use__MNL_INSTAL'] <= end)]

    # based on time window, calculate appropriate age of pipes (select beginning year of time frame --> ex: 2009, 2010, 2011
    #       subtract install year from 2009) - MNL_INSTAL col
    # -- will create negative numbers
    train_df['AGE'] = start - train_df['DWW_Mainlines__Permitted_Use__MNL_INSTAL']

    return train_df


In [78]:
train_df = get_data(dummy_df, 2009, 2011)
# train_df.index

In [79]:
# trains
# tests
# move over
# rinse & repeat until 2019
def split_df(df):
    """
    """    
    df = df.dropna()
    feature_df = df.drop(['TARGET', 'DATE'], axis=1)
    target_df = df[['TARGET']]

    return feature_df, target_df

def train_v1(df, model):
    """
    Takes in a dataframe of interest and a model, and trains the model.
    Model likely needs to be one imported from sklearn so it is able to
    handle dataframes as input data
    """
    feature, target = split_df(df)
    clf = model #clf is for for classification
    clf.fit(feature, target)

    return clf

In [80]:
# Training data with ExtraTreesClassifier and split data
extra_tree_model = train_v1(train_df)
feature, target = split_df(train_df)

# running a prediction model on training set
training_pred = extra_tree_model.predict(feature)
acc = accuracy_score(y_true=target, y_pred=training_pred)
acc

# confusion matrix for training
# confusion_matrix(y_true=target, y_pred=training_pred)

TypeError: train_v1() missing 1 required positional argument: 'model'

In [117]:
def main_function(dummy_df, start, end, model):
    """
    Takes in dummy dataframe and runs all functions based on given year ranges.
    Prints out year and accuracy per time range (3 years ranges)
    """

    full_results = {} # Dictionary for storing all the models and their data

    for i in range(start, end - 4):

        

        start_train = i
        end_train = i + 2
        df = get_data(dummy_df, start_train, end_train)
        feature_train, target_train = split_df(df)
        clf = train_v1(df, model)

        training_pred = clf.predict(feature_train)
        #print(start_train, "-", end_train, ": ")#, accuracy_score(y_true=target, y_pred=training_pred)) 

        # Test
        start_test = i + 3
        end_test = start_test + 2
        test = get_data(dummy_df, start_test, end_test)
        feature_test, target_test = split_df(test)
        testing_pred = clf.predict(feature_test)
        #testing_proba_preds = clf.predict_proba(feature_test)[::,1]
        #print(start_test, "-", end_test, ": ", accuracy_score(y_true=target_test, y_pred=testing_pred))
        
        print(start_train, "-", end_test, ": ")#, accuracy_score(y_true=target, y_pred=training_pred)) 


        # Print out break and non break accuracy's rather total full accuracy
        cm = confusion_matrix(y_true=target_test, y_pred=testing_pred)
        # nonbreak_acc = cm[0,0] / (cm[0,0] + cm[0,1])
        # print('Non-break accuracy: ', nonbreak_acc)
        # break_acc = cm[1,1] / (cm[1, 0] + cm[1,1])
        # print('Break accuracy: ', break_acc)

        #print out metrics
        # precision, recall, f1, support = precision_recall_fscore_support(y_true=target_test, y_pred=testing_pred)
        # print('Precision: ', precision)
        # print('Recall: ', recall)
        # print('F1 Score: ', f1)
        # print('Support: ', support)

        precision = precision_score(y_true=target_test, y_pred=testing_pred)
        recall = recall_score(y_true=target_test, y_pred=testing_pred)
        bal_acc = balanced_accuracy_score(y_true=target_test, y_pred=testing_pred)
        print('precision: ', precision)
        print('recall: ', recall)
        print('Balanced accuracy: ', bal_acc)
        
        print()

        model_results = {
            'model': clf,
            'start_year': start_test,
            'end_year': end_test,
            'test_df': feature_test, # For eventual SHAP calculations
            'target_values': target_test,
            'test_predictions': testing_pred,
            'confusion_matrix': cm

            }

        full_results['model_' + str(start_test) + '-' + str(end_test)] = model_results
    
    return full_results

In [147]:
gbt = GradientBoostingClassifier() # ~58% accuracy
ada = AdaBoostClassifier() # 45-52% accuracy
#stc = StackingClassifier()
rf = RandomForestClassifier()
bag = BaggingClassifier()
etc = ExtraTreesClassifier()#max_depth=100, max_features='log2', n_estimators=100) # ~58% accuracy

results = main_function(dummy_df, 2009, 2019, cv_model.best_estimator_)

2009 - 2014 : 
precision:  1.0
recall:  0.5988700564971752
Balanced accuracy:  0.7994350282485876

2010 - 2015 : 
precision:  1.0
recall:  0.544839255499154
Balanced accuracy:  0.772419627749577

2011 - 2016 : 
precision:  1.0
recall:  0.5924092409240924
Balanced accuracy:  0.7962046204620462

2012 - 2017 : 
precision:  1.0
recall:  0.5567484662576687
Balanced accuracy:  0.7783742331288344

2013 - 2018 : 
precision:  1.0
recall:  0.586490939044481
Balanced accuracy:  0.7932454695222405

2014 - 2019 : 
precision:  1.0
recall:  0.571917808219178
Balanced accuracy:  0.785958904109589



In [101]:
print(results.keys())

dict_keys(['model_2012-2014', 'model_2013-2015', 'model_2014-2016', 'model_2015-2017', 'model_2016-2018', 'model_2017-2019'])


In [144]:
param_grid = {'max_depth': [None, 100, 300, 500],
    'max_features': ['log2', 'sqrt', 'auto'],
    'n_estimators': [100, 200, 300, 400, 500]
    }


cv_model = RandomizedSearchCV(estimator=etc, param_distributions=param_grid, scoring='recall', n_iter=10, n_jobs=-1)

X_df, y_df = split_df(train_df)
cv_model.fit(X_df, y_df)

RandomizedSearchCV(estimator=ExtraTreesClassifier(), n_jobs=-1,
                   param_distributions={'max_depth': [None, 100, 300, 500],
                                        'max_features': ['log2', 'sqrt',
                                                         'auto'],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500]},
                   scoring='recall')

In [145]:
cv_model.best_params_

{'n_estimators': 400, 'max_features': 'auto', 'max_depth': 300}

In [146]:
cv_model.best_score_

0.5418181818181819

Look at SHAP values, and feature importance values, to determine what is influecing the model most

In [149]:
explainer = shap.TreeExplainer(cv_model.best_estimator_)
shap_values = explainer.shap_values(X_df)

In [ ]:

class_inds = np.argsort([-np.abs(shap_values[i]).mean() for i in range(len(shap_values))])
cmap = plt_colors.ListedColormap(np.array(colors)[class_inds])

In [ ]:
shap.summary_plot(shap_values, X_df, class_names=np.array(y_df.values.unique()), max_display=15, title='Total SHAP Values')